# Objetivo

Predecir el género de canciones a partir de su letra. Para ello usaremos un modelo transformers basado en BERT.

# Obtención del dataset

Primero de todos importamos las librerías necesarias para trabajar.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
import re
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import torch

In [2]:
# Detectar device (GPU si está disponible, si no CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando device:", device)

Usando device: cuda


In [3]:
from google.colab import drive  # API para montar Google Drive en Colab
drive.mount('/content/drive')  # montar drive en el directorio /content/drive (Colab)

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/spotify_dataset_5000.csv'
df = pd.read_csv(path)
df = df[['text', 'Genre']]

# Preprocesamiento del dataset

## Limpieza

Observamos que estamos frente a un problema multiclase. Por tanto, lo primero que vamos a hacer es ajustar la columna "Genre". Como viene en formato string, lo pasaremos a formato lista.

In [5]:
# Convertir la columna 'Genre' de string ("a, b") a lista ['a','b']
df["genre_list"] = df["Genre"].apply(lambda x: [g.strip() for g in x.split(",")])  # split y strip por cada género
df.drop(columns=["Genre"], inplace=True)  # eliminar la columna original ya convertida

Una vez hecho esto, vamos a tratar las letras de las canciones. Vamos a eliminar los corchetes y la información que aparece dentro de ellos.

In [6]:
def limpiar_corchetes(texto):  # definir función para eliminar contenido entre corchetes
    # Esto elimina cualquier cosa entre [], incluyendo los corchetes
    return re.sub(r"\[.*?\]", "", texto).strip()  # sustituir por vacío y quitar espacios exteriores

# Aplicar la limpieza a la columna de texto y guardar en 'clean_text'
df['clean_text'] = df['text'].apply(limpiar_corchetes)  # limpiar corchetes de cada letra
df.drop(columns=['text'], inplace=True)  # eliminar la columna original de texto

Ahora vamos a pasar a binarizar la variable Genre.

In [7]:
mlb = MultiLabelBinarizer()  # crear binarizador para etiquetas multilabel
labels = mlb.fit_transform(df['genre_list'])  # ajustar a las listas de géneros y transformar a matriz binaria
df['labels_binarios'] = list(labels)  # almacenar vectores binarios como lista en DataFrame
generos_unicos = mlb.classes_  # obtener nombres de géneros únicos
num_labels = len(generos_unicos)  # contar cuántas etiquetas existen

In [11]:
print(f"Géneros únicos: {generos_unicos}")  # mostrar la lista de géneros
print(f"Número de géneros únicos: {num_labels}")  # mostrar el número de etiquetas disponibles
print(labels)

Géneros únicos: ['acoustic' 'alt-country' 'alternative' 'alternative rock' 'ambient'
 'black metal' 'blues' 'britpop' 'chillwave' 'christian' 'classic rock'
 'classical' 'cloud rap' 'comedy' 'country' 'dance' 'dancehall'
 'death metal' 'deathcore' 'disco' 'doom metal' 'dream pop'
 'drum and bass' 'dub' 'dubstep' 'electro' 'electronic' 'electropop' 'emo'
 'emo rap' 'experimental' 'folk' 'funk' 'garage rock' 'gospel' 'grime'
 'grunge' 'hard rock' 'hardcore' 'heavy metal' 'hip hop' 'hip-hop' 'house'
 'indie' 'indie pop' 'indie rock' 'industrial' 'j-pop' 'jazz' 'k-pop'
 'latin' 'lo-fi' 'math rock' 'melodic death metal' 'metal' 'metalcore'
 'new wave' 'nu metal' 'pop' 'pop punk' 'pop rock' 'post-hardcore'
 'post-punk' 'power metal' 'progressive metal' 'progressive rock'
 'psychedelic' 'psychedelic rock' 'punk' 'punk rock' 'rap' 'reggae' 'rnb'
 'rock' 'screamo' 'shoegaze' 'soul' 'soundtrack' 'swing' 'synthpop'
 'techno' 'thrash metal' 'trance' 'trap' 'trip-hop' 'worship']
Número de géneros ú

In [10]:
df_final = df[['clean_text', 'labels_binarios']].copy()  # seleccionar solo texto limpio y labels binarios y crear copia segura
df_final.head()

,clean_text,labels_binarios
0,Lyrics/Music Yellen/Pokrass Why are grown up p...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"Sharing's good, sharing's fine But no one w...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2,"Let it go, let it go Can’t hold it back anymor...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"(Madara) J'S, J'S If your friend gets packed...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"""Love? You know, what do you know about love? ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


Por último vamos a borrar las canciones que aparecen repetidas

In [12]:
df_final.drop_duplicates(subset=['clean_text'], inplace=True)  # eliminar filas con letras duplicadas para evitar sesgo por duplicados

## Tokenizador

Dividimos el dataset en entrenamiento, validación y test en una proporción de 80-20-20

In [ ]:
train_val_df, test_df = train_test_split(df_final, test_size=0.2, random_state=42)  # separar 20% para test
train_df , val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)  # dividir el 80% restante en train (60%) y val (20%)

Convertimos a formato Dataset de HuggingFace

In [ ]:
import gc

# Convertimos a Dataset de HF
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Limpieza de RAM: borramos los dataframes de Pandas que ya no sirven y ocupan espacio
del df, df_final, train_df, val_df, test_df, train_val_df
gc.collect() # Forzamos al recolector de basura de Python a liberar memoria
print("Memoria RAM liberada. Listo para tokenizar.")

Memoria RAM liberada. Listo para tokenizar.


Cargamos el tokenizador

In [ ]:
MODEL_NAME = "distilbert-base-multilingual-cased"  # nombre del modelo preentrenado a usar
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  # cargar el tokenizador correspondiente al modelo

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Función de preprocesamiento: tokenizamos y convertimos los labels a float

In [ ]:
def preprocess_data(example):  # función que tokeniza y prepara labels para el modelo
    encoding = tokenizer(
        example['clean_text'],  # texto limpio a tokenizar
        padding='max_length',  # rellenar hasta la longitud máxima
        truncation=True,  # truncar si excede max_length
        max_length=512  # longitud máxima de tokens
    )  # resultado: diccionario con input_ids, attention_mask, etc.
    encoding['labels'] = [np.array(label, dtype=np.float32) for label in example['labels_binarios']]  # convertir labels a float32
    return encoding  # devolver diccionario con inputs y labels

Aplicamos la función a los datasets

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_data, batched=True,
                                            remove_columns=['clean_text', 'labels_binarios', '__index_level_0__'])  # aplicar preprocesado y eliminar columnas originales en train
tokenized_val_dataset = val_dataset.map(preprocess_data, batched=True,
                                        remove_columns=['clean_text', 'labels_binarios', '__index_level_0__'])  # aplicar a validación
tokenized_test_dataset = test_dataset.map(preprocess_data, batched=True,
                                        remove_columns=['clean_text', 'labels_binarios', '__index_level_0__'])  # aplicar a test

Map:   0%|          | 0/2954 [00:00<?, ? examples/s]

Map:   0%|          | 0/985 [00:00<?, ? examples/s]

Map:   0%|          | 0/985 [00:00<?, ? examples/s]

Mapeo de etiquetas y modelo

In [ ]:
# Mapear índices a etiquetas y viceversa para el modelo
id2label = {i: label for i, label in enumerate(generos_unicos)}  # id -> nombre etiqueta
label2id = {label: i for i, label in enumerate(generos_unicos)}  # nombre etiqueta -> id

# Cargar modelo preentrenado para clasificación de secuencias (ajustar para multilabel)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels, # El número de géneros únicos
    problem_type="multi_label_classification", # indicar que es multilabel
    id2label=id2label,
    label2id=label2id
)  # devuelve un modelo listo para fine-tuning en multilabel

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Configuramos el entrenamiento: AdamW y Early Stopping

In [ ]:
# Definimos las métricas de evaluación para multi-label
def compute_metrics(eval_pred):
    logits, labels = eval_pred  # logits sin normalizar y labels verdaderas

    pred_probs = 1 / (1 + np.exp(-logits))  # Aplicar la función sigmoide para obtener probabilidades
    preds = (pred_probs >= 0.2).astype(int)  # Umbral por defecto (0.2) para convertir probabilidades a 0/1

    f1_micro = f1_score(labels, preds, average='micro', zero_division=0)  # F1 micro (suma sobre clases)
    f1_macro = f1_score(labels, preds, average='macro', zero_division=0)  # F1 macro (promedio por clase)
    accuracy = accuracy_score(labels, preds)  # accuracy binaria por etiqueta
    roc_auc = roc_auc_score(labels, pred_probs, average='micro')  # ROC AUC usando probabilidades

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'accuracy': accuracy,
        'roc_auc': roc_auc
    }

# Callback de early stopping para detener entrenamiento si no hay mejora
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3
)

# Configuramos argumentos de entrenamiento
training_args = TrainingArguments(
    # 1. SEGURIDAD: Guardamos en Drive por si Colab se desconecta
    output_dir="/content/drive/MyDrive/checkpoints_spotify_50k",
    overwrite_output_dir=False,

    # 2. VELOCIDAD: Evaluamos cada 500 pasos (aprox cada 20-30 min)
    save_strategy="steps", # Aquí sería mejor poner epochs. Pendiente de cambiar.
    save_steps=500,
    eval_strategy="steps", # Aquí sería mejor poner epochs. Pendiente de cambiar
    eval_steps=500,
    save_total_limit=2,           # Solo guardamos los 2 últimos para no llenar Drive

    # Configuración de Hardware
    learning_rate=2e-5,
    fp16=True,                    # Vital para memoria GPU
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16, # Validación más rápida

    # Configuración del Modelo
    load_best_model_at_end=True,
    num_train_epochs=3,           # 3 épocas suelen sobrar para 50k
    weight_decay=0.01,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    logging_steps=100,            # Logs frecuentes para ver que no se cuelga
    report_to=["none"]
)

# El trainer se mantiene igual
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

# Entrenar

In [ ]:
# Entrenamos
trainer.train()  # ejecutar el loop de entrenamiento

print("\nEntrenamiento completado.")

# Evaluar el mejor modelo (cargado automáticamente)
eval_results = trainer.evaluate()  # obtener métricas en el conjunto de validación
print("Resultados finales de la evaluación:")
print(eval_results)  # mostrar las métricas

# Guardar el modelo final y el tokenizador
trainer.save_model("./mi_modelo_genero_multilabel")  # guardar pesos y configuración del modelo
tokenizer.save_pretrained("./mi_modelo_genero_multilabel")  # guardar tokenizador para inferencia posterior

Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy,Roc Auc
500,0.078700,0.073838,0.386753,0.007913,0.454822,0.830463



Entrenamiento completado.


Resultados finales de la evaluación:
{'eval_loss': 0.07383794337511063, 'eval_f1_micro': 0.3867529501332318, 'eval_f1_macro': 0.007912895839499058, 'eval_accuracy': 0.4548223350253807, 'eval_roc_auc': 0.8304634098697414, 'eval_runtime': 4.6434, 'eval_samples_per_second': 212.127, 'eval_steps_per_second': 13.352, 'epoch': 3.0}


('./mi_modelo_genero_multilabel/tokenizer_config.json',
 './mi_modelo_genero_multilabel/special_tokens_map.json',
 './mi_modelo_genero_multilabel/vocab.txt',
 './mi_modelo_genero_multilabel/added_tokens.json',
 './mi_modelo_genero_multilabel/tokenizer.json')

# Predicciones

In [ ]:
# 1. Generar predicciones sobre el conjunto de test
output_predictions = trainer.predict(tokenized_test_dataset)

# 2. Ver las métricas
print("Métricas Finales en Test")
for key, value in output_predictions.metrics.items():
    print(f"{key}: {value:.4f}")

# 3. Visualizar ejemplos reales (Decodificar de [0,1,0] a ['Pop', 'Rock'])
from scipy.special import expit

# Obtener probabilidades y binarizar
probs = expit(output_predictions.predictions)
preds_binary = (probs >= 0.2).astype(int)

# Función auxiliar para traducir vector binario a lista de textos
def decode_labels(binary_row, id2label_map):
    return [id2label_map[i] for i, val in enumerate(binary_row) if val == 1]

print("\n Comparativa Visual (Primeras 5 canciones)")
for i in range(5):
    # Etiquetas Reales
    real_genres = decode_labels(output_predictions.label_ids[i], id2label)
    # Predicciones
    pred_genres = decode_labels(preds_binary[i], id2label)

    print(f"Canción {i+1}:")
    print(f"  🟢 Real:     {real_genres}")
    print(f"  🤖 Predicho: {pred_genres}")
    print("-" * 50)

Métricas finales en el Test Set (con umbral 0.5, engañosas):
{'test_loss': 0.07375268638134003, 'test_f1_micro': 0.39096045197740115, 'test_f1_macro': 0.008025111331024244, 'test_accuracy': 0.46598984771573604, 'test_roc_auc': 0.837548836575404, 'test_runtime': 8.0748, 'test_samples_per_second': 121.985, 'test_steps_per_second': 7.678}

--- MÉTRICAS CON UMBRAL = 0.2 ---
F1-Macro (Nuevo): 0.008025111331024244
F1-Micro (Nuevo): 0.39096045197740115
Accuracy (Nuevo): 0.46598984771573604


In [ ]:
from huggingface_hub import login

# Ejecuta esto y pega tu token TOKEN DE ESCRITURA (WRITE) cuando te lo pida
login(add_to_git_credential=False)

In [ ]:
from huggingface_hub import login

# 1. Loguearse (te pedirá el token)
login()

# 2. Definir el nombre de tu repositorio en el Hub
# Formato: "tu_usuario/nombre_del_modelo"
repo_id = "Juanpeg1729/genre-classifier"

# 3. Subir el modelo y el tokenizador
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"¡Modelo subido exitosamente a https://huggingface.co/{repo_id}!")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ctt5qem/model.safetensors:   0%|          | 10.9kB /  542MB            

README.md: 0.00B [00:00, ?B/s]

¡Modelo subido exitosamente a https://huggingface.co/Juanpeg1729/genre-classifier!
